In [23]:
import os
from datetime import datetime

import pandas as pd
import databricks.koalas as ks

In [ ]:
# get data first
BUCKET_ID = 'bitcoin_bucket_20200308'
BUCKET_URL = f'gs://{BUCKET_ID}'
os.system(f'gsutil -m cp -r {BUCKET_URL} .')

#!gsutil -m cp -r gs://bitcoin_bucket_20200308 .

In [2]:
def build_frame_pandas(base_path):
    df = pd.DataFrame([])
    for filename in os.listdir(base_path):
        full_path = f'{base_path}/{filename}'
        name = filename.split('-')[0]
        new = pd.read_csv(full_path)
        new['open_time'] = pd.to_datetime(
            new['open_time'],
            unit='ms'
        )
        new = new.set_index('open_time', drop=True)
        new[name] = (new['open'] + new['close']) / 2
        df = df.join(new[name], how='outer')
    return df

In [3]:
def build_frame_koalas(base_path):
    first_run = True
    for filename in os.listdir(base_path):
        full_path = f'{base_path}/{filename}'
        name = filename.split('-')[0]
        new = ks.read_csv(full_path)
        new['open_time'] = ks.to_datetime(
            new['open_time'],
            unit='ms'
        )
        new = new.set_index('open_time', drop=True)
        new[name] = (new['open'] + new['close']) / 2
        if first_run:
            df = ks.DataFrame(new[name])
            first_run = False
        else:
            df = df.join(new[name], how='outer')
    return df

In [ ]:
start = datetime.now()
df = build_frame_koalas()

print(datetime.now() - start)
print(df.info())
print(df.resample('1min').asfreq().ffill().info())